In [1]:
import pandas as pd
import numpy as np

### Сырой DataFrame

In [2]:
df_raw = pd.read_csv("../../resources/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
df_raw.columns=["unixtimestamp", "open", "high","low","close", "volume_btc","volume_usd","weighted_price"]

### Чистка данных

In [3]:
df = df_raw.dropna()
df = df[df["volume_btc"] > 0]
df.insert(loc=0, column='timestamp', value=pd.to_datetime(df['unixtimestamp'],unit = 's'))
df.reset_index(inplace=True, drop=True)

In [4]:
df.head()

,timestamp,unixtimestamp,open,high,low,close,volume_btc,volume_usd,weighted_price
0,2011-12-31 07:52:00,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,2011-12-31 15:50:00,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
2,2011-12-31 16:59:00,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
3,2011-12-31 17:00:00,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
4,2012-01-01 04:16:00,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000


### Расширение данных

In [5]:
df_shifted = df[["open", "high", "low", "close", "volume_btc", "volume_usd", "weighted_price"]].shift(1)
df[["open_prev1", "high_prev1", "low_prev1", "close_prev1", "volume_btc_prev1", "volume_usd_prev1", "weighted_price_prev1"]] = df_shifted
df[["open_rel", "high_rel", "low_rel", "close_rel", "volume_btc_rel", "volume_usd_rel", "weighted_price_rel"]] = \
    df[["open", "high", "low", "close", "volume_btc", "volume_usd", "weighted_price"]] / df_shifted

In [6]:
df["sma14"] = df[["open"]].rolling(14).mean()
df["sma30"] = df[["open"]].rolling(30).mean()

In [24]:
df[["open", "close", "close_prev1"]].values.reshape((-1,3))

array([[4.390000e+00, 4.390000e+00,          nan],
       [4.390000e+00, 4.390000e+00, 4.390000e+00],
       [4.500000e+00, 4.570000e+00, 4.390000e+00],
       ...,
       [5.869343e+04, 5.872384e+04, 5.868581e+04],
       [5.874218e+04, 5.876059e+04, 5.872384e+04],
       [5.876775e+04, 5.877818e+04, 5.876059e+04]])

In [27]:
def obv(acc, ser):
    print(acc, ser)
    vol=ser[0]
    close=ser[1]
    close_prev1=ser[2]
    if close > close_prev1:
        return acc + vol
    elif close < close_prev1:
        return acc - vol
    else:
        return acc

def obv_acc(ser):
    print(ser)
    obv_op = np.frompyfunc(obv, 2, 1)
    return obv_op.reduce(ser, 1, dtype=np.ndarray)

In [28]:
obv_acc(df[["open", "close", "close_prev1"]].values.reshape((-1,3)))

[[4.390000e+00 4.390000e+00          nan]
 [4.390000e+00 4.390000e+00 4.390000e+00]
 [4.500000e+00 4.570000e+00 4.390000e+00]
 ...
 [5.869343e+04 5.872384e+04 5.868581e+04]
 [5.874218e+04 5.876059e+04 5.872384e+04]
 [5.876775e+04 5.877818e+04 5.876059e+04]]
4.39 4.39


TypeError: 'float' object is not subscriptable

In [30]:
np.add.reduce(df[["open", "close", "close_prev1"]].values.reshape((-1,3)), axis=1)

array([          nan, 1.3170000e+01, 1.3460000e+01, ..., 1.7610308e+05,
       1.7622661e+05, 1.7630652e+05])

In [56]:
df.drop(0, inplace=True)